In [2]:
import pandas as pd
import numpy as np
import time

In [7]:
pickup_data = pd.read_pickle('arrival_and_dropoff_distributions')
trip_time_data = pd.read_csv('trip_time_means.csv', index_col = 'pulocationid')

In [8]:
pickup_data

pulocationid
1      ([0.0, 0.0, 0.0, 0.03225806451612903, 0.0, 0.0...
2      ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
3      ([27.161290322580644, 19.580645161290324, 15.1...
4      ([71.41935483870968, 55.96774193548387, 41.387...
5      ([3.0, 2.967741935483871, 2.774193548387097, 2...
                             ...                        
260    ([74.58064516129032, 62.38709677419355, 53.387...
261    ([46.483870967741936, 25.741935483870968, 18.1...
262    ([37.516129032258064, 20.774193548387096, 16.6...
263    ([73.2258064516129, 54.096774193548384, 40.741...
265    ([0.7741935483870968, 0.7741935483870968, 0.70...
Length: 262, dtype: object

In [9]:
trip_time_data

,1,2,3,4,5,6,7,8,9,10,...,247,248,249,250,251,252,253,254,255,256
pulocationid,,,,,,,,,,,,,,,,,,,,,
1,19.037500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,54.540476,NaN,5.167347,43.009259,NaN,NaN,26.164216,NaN,21.255556,28.980556,...,25.146182,14.203064,39.903333,12.393053,NaN,20.235000,NaN,8.283445,36.853333,50.141667
4,38.440143,NaN,36.005556,8.353476,39.816667,30.175000,25.623986,NaN,32.711111,34.955556,...,25.487634,27.186667,14.388520,29.864583,NaN,29.752778,NaN,32.131944,17.887951,15.287944
5,26.337016,NaN,NaN,NaN,6.377981,26.475439,NaN,NaN,NaN,63.383333,...,NaN,NaN,50.329167,NaN,19.100366,NaN,NaN,NaN,41.983333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,50.356373,NaN,29.951515,24.593915,NaN,NaN,9.541995,12.531944,21.596930,27.229938,...,20.965068,19.543243,29.454167,21.740873,NaN,18.506399,12.72381,28.170290,16.476681,17.322406
261,38.053702,NaN,39.591667,14.067819,38.878571,22.952083,29.628744,NaN,35.214583,51.258974,...,28.823718,47.005556,11.600757,37.881373,27.261667,49.226111,NaN,39.840000,23.663167,21.626105
262,45.954111,NaN,24.680556,15.973396,NaN,32.400000,19.359014,NaN,36.588095,38.552083,...,15.308594,20.514167,25.081425,25.472917,36.216667,24.427778,NaN,27.887500,26.464657,26.170968
